In [1]:
import practice2
import matplotlib.pyplot as plt
import pandas as pd
import math
import time

In [2]:
directory = "Practice_03_data/"
list_data = practice2.list_file_data(directory)
filename = str(directory+list_data[0])

## Exercise 2: Collection Statistics 

In [3]:
pl_1, lt_1 = practice2.text_mining(filename,False)

40.80051851272583


In [28]:
stat_1 = practice2.get_statistics(pl_1, lt_1)

In [5]:
dl = pd.DataFrame(stat_1['df'],columns=['Document numbers', 'DL'])
tl = pd.DataFrame(stat_1['tl'],columns=['Terms', 'tl'])

In [6]:
c_freq_l = list()
for k,v in stat_1['colec_freq'].items():
    c_freq_l.append((k,v[0][0],v[0][1]))

In [7]:
c_freq = pd.DataFrame(c_freq_l,columns=['Terms', 'freq','Df'])

In [8]:
tl_red = tl[:100]
c_f_red = c_freq[:150]

In [9]:
writer = pd.ExcelWriter('statistique.xlsx')
dl.to_excel(writer,'Document lenght')
tl_red.to_excel(writer,'Terms lenght')
c_f_red.to_excel(writer,'Collection frequency terms')
writer.save()

## Exercise 3: Collection Statistics using stop-words and stemmer 

In [4]:
pl_2, lt_2 = practice2.text_mining(filename,True)

1441.4933235645294


In [5]:
stat_2 = practice2.get_statistics(pl_2,lt_2)

In [6]:
dl = stat_2['df'] 
#print(dl)
word_total = sum([dl[x] for x in dl.keys()])# <--- somme de tous les termes dans toutes la collections

6604890

In [7]:
dl = pd.DataFrame(stat_2['df'],columns=['Document numbers', 'DL'])
tl = pd.DataFrame(stat_2['tl'],columns=['Terms', 'tl'])

c_freq_l = list()
i=0
for k,v in stat_2['colec_freq'].items():
    i+=1
    if i==200: break
    c_freq_l.append((k,v[0][0],v[0][1]))

c_freq = pd.DataFrame(c_freq_l,columns=['Terms', 'freq','Df'])

tl_red = tl[:100]
c_f_red = c_freq[:150]

writer = pd.ExcelWriter('statistique_stopword_stemming.xlsx')
dl.to_excel(writer,'Document lenght')
tl_red.to_excel(writer,'Terms lenght')
c_f_red.to_excel(writer,'Collection frequency terms')
writer.save()

## Exercise 4: SMART ltn weighting 

In [8]:
def smart_ltn(posting_list,n, df_t):
    df = 0
    #(1+LOG(tf))*LOG(n/df)
    #tf is the frequence of term on a document
    #df number of document where the term appear
    ltn_value = {} # key doc number, value (terme, ltn)
    for term, value in posting_list.items():
        dl = df_t[term]
        df = dl[0][1] # nombre de doc dans lequel le terme apparait sur toute la collection
        for v in value:
            tf = v[1]
            ltn = (1+math.log10(tf))*(math.log10(n/df))
            ltn_value.setdefault(v[0],[]).append((term,round(ltn,4)))
    return ltn_value

s_ltn = smart_ltn(pl_2, stat_2['n_doc'],stat_2['colec_freq'])

## Exercise 5: Ranked Retrieval (ltn weighting) 

In [9]:
query = "web ranking scoring algorithm"
query = practice2.clean(query, True)
# Doc = score(sucle(ltn(term)))
def rsv_score(query,wf):
    rsv_doc = {}       

    for term in query.split():
        for doc, values in wf.items():
            for v in values:
                if term == v[0]:
                    rsv_doc.setdefault(doc,[]).append(v[1]) 
                else: continue
                    
    rsv = [(sum(wf_list),doc) for doc,wf_list in rsv_doc.items()]
    return rsv

#rsv = practice2.scoring_doc(practice2.clean(query, True), posting_list, list_terms)

In [10]:
rsv = rsv_score(query,s_ltn)
score = sorted(rsv, reverse=True) # To get the score descending
score = [(doc, round(ltn,4)) for ltn, doc in score]
score[:10]

[('23724', 6.8495),
 ('70421', 6.4105),
 ('1482394', 6.2382),
 ('448834', 6.1813),
 ('12431', 5.7765),
 ('462448', 5.5505),
 ('207747', 5.4624),
 ('1793571', 5.435),
 ('587642', 5.4273),
 ('47293', 5.3256)]

## Exercise 6: SMART ltc weighting 
 - Compute a weighted index based on SMART ltc weighting function. 

In [11]:
def smart_ltc(smart_ltn):
    # ltn = doc:(term, ltn)
    #  (ltn_term/sqrt(sum(each pow(ltn_term,2))on a document))
    ltc_value = {}
    for doc, value in smart_ltn.items():
        pow_ltn_term = [pow(v[1],2) for v in value]
        normalize = math.sqrt(sum(pow_ltn_term))
        for v in value:
            ltc_value.setdefault(doc,[]).append((v[0],round((v[1]/normalize),4)))
    return ltc_value


## Exercise 7: Ranked Retrieval (ltc weighting) 
Compute the score of each document for the query  « web ranking scoring algorithm », using the index based 
on SMART ltc weighting function. Print the list of the ten most relevant documents, and their relevance score

In [12]:
s_ltc = smart_ltc(s_ltn)
rsv_c = rsv_score(query,s_ltc)
score_c = sorted(rsv_c, reverse=True)
score_c = [(doc, round(ltc,4)) for ltc, doc in score_c]
score_c[:10]

[('18096221', 0.2324),
 ('2363262', 0.2253),
 ('6422823', 0.2202),
 ('1395835', 0.2158),
 ('8080270', 0.2123),
 ('10416781', 0.2028),
 ('4839372', 0.2016),
 ('1851223', 0.2007),
 ('19413518', 0.2007),
 ('2851233', 0.1932)]

## Exercise 8: BM25 weighting 
Compute a weighted index based on BM25 weighting function. 

In [13]:
def tf_part(posting_list,dl,n):
    #(tf*(k+1))/(k*((1-b)+b*(dl/avdl))+tf)
    #[  1     ]             [   2   ]
    #              [      3         ]
    #            [         4            ]   
    k = 1.0
    b = 0.5
    dl_ = sum([dl[x] for x in dl.keys()])
    doc_len = 0
    avdl = dl_/n
    #print(avdl)
    #avdl = 20
    tf_part_val = {}
    for term, value in posting_list.items():
        for v in value:
            #if not term in v
            tf = v[1]
            doc_len = dl[v[0]]
            #doc_len = [doc_l for docno,doc_l in dl if docno==v[0]]
            bloc_1 = tf*(k+1)
            bloc_2 = doc_len/avdl
            bloc_3 = k * ((1-b) + b * bloc_2)
            bloc_4 = bloc_3+tf       
            tf_part_val.setdefault(v[0],[]).append((term,bloc_1/bloc_4)) 
    return tf_part_val

In [12]:
#dl = stat_2['df']
#tf_part(pl_2, stat_2['df'],stat_2['n_doc'])

In [14]:
def idf_part(posting_list,df_,n):
    #log((n-df+0.5)/(df+0.5))
    #    [   1    ] [  2   ]
    idf_part_val ={}
    for term, value in posting_list.items():
        df = df_[term]
        df = df[0][1]
        bloc_1 = abs(n-df+0.5)
        bloc_2 = df + 0.5
        idf_part_val.setdefault(term,[]).append(math.log10(bloc_1/bloc_2))
    return idf_part_val


In [55]:
#idf_part(pl_2,stat_2['colec_freq'], stat_2['n_doc'])

In [15]:
def bm25(posting_list, stat):
    tf_part_ = tf_part(posting_list,stat['df'],stat['n_doc'])

    idf_part_ = idf_part(posting_list,stat['colec_freq'], stat['n_doc'])
    bm25_val = {}
    # doc 1 : ('a' , tf_part_[a]*idf[a])
    for doc, tf_value in tf_part_.items():
        for tf in tf_value:
            bm25_val.setdefault(doc,[]).append((tf[0], tf[1]*idf_part_[tf[0]][0]))     
    return bm25_val
    

## Exercise 9: Ranked Retrieval (BM25 weighting) 

In [16]:
bm25_val = bm25(pl_2,stat_2)

In [17]:
bm25_score= rsv_score(query,bm25_val)
def reverse_score(score_result):
    score = sorted(score_result, reverse=False)
    score = [(doc, result) for result, doc in score]
    return score

[('187326', 0.16492234183028834),
 ('21525', 0.17389084150482922),
 ('2095669', 0.19371905274607265),
 ('16818407', 0.2082106731276129),
 ('58353', 0.22617769907954288),
 ('8177460', 0.24622534467780063),
 ('48728', 0.2480262101148634),
 ('1990194', 0.25843030456337124),
 ('1649530', 0.26514453611936334),
 ('1347979', 0.26676664209739426)]

In [36]:
## 2009011 Q0 15343730 1 0.011317152113888167 Mathias /article[1]
score = sorted(score,reverse=False)
x = score
query = '2009011 olive oil health benefit'

rsv_r = [[query.split()[0],'Q0', x[i][0],i+1,x[i][1], 'Dounia', '/article[1]'] for i in range(len(x))]

In [37]:
def weinting_function(pl, stat):
    ltn = smart_ltn(pl,stat['n_doc'], stat['colec_freq'])
    ltc = smart_ltc(pl, ltn)
    bm25_val = bm25(pl_2,stat_2)
    
    return ltn,ltc,bm25_val

[['2009011', 'Q0', '10003934', 1, 1.1477, 'Dounia', '/article[1]'],
 ['2009011', 'Q0', '10008302', 2, 1.2283, 'Dounia', '/article[1]'],
 ['2009011', 'Q0', '10013', 3, 2.6459, 'Dounia', '/article[1]'],
 ['2009011', 'Q0', '10013985', 4, 3.0603, 'Dounia', '/article[1]'],
 ['2009011', 'Q0', '10018', 5, 1.1477, 'Dounia', '/article[1]'],
 ['2009011', 'Q0', '1002307', 6, 0.8397, 'Dounia', '/article[1]'],
 ['2009011', 'Q0', '10037185', 7, 2.5436, 'Dounia', '/article[1]'],
 ['2009011', 'Q0', '10045226', 8, 0.6454, 'Dounia', '/article[1]'],
 ['2009011', 'Q0', '1004679', 9, 3.1454, 'Dounia', '/article[1]'],
 ['2009011', 'Q0', '1004743', 10, 1.8545, 'Dounia', '/article[1]'],
 ['2009011', 'Q0', '10048713', 11, 0.6454, 'Dounia', '/article[1]'],
 ['2009011', 'Q0', '100558', 12, 1.5795, 'Dounia', '/article[1]'],
 ['2009011', 'Q0', '10056712', 13, 1.0051, 'Dounia', '/article[1]'],
 ['2009011', 'Q0', '10057294', 14, 2.2352, 'Dounia', '/article[1]'],
 ['2009011', 'Q0', '1005923', 15, 2.0755, 'Dounia', '/

In [42]:
 = 

"[['2009011', 'Q0', '10003934', 1, 1.1477, 'Dounia', '/article[1]'], ['2009011', 'Q0', '10008302', 2, 1.2283, 'Dounia', '/article[1]'], ['2009011', 'Q0', '10013', 3, 2.6459, 'Dounia', '/article[1]'], ['2009011', 'Q0', '10013985', 4, 3.0603, 'Dounia', '/article[1]'], ['2009011', 'Q0', '10018', 5, 1.1477, 'Dounia', '/article[1]'], ['2009011', 'Q0', '1002307', 6, 0.8397, 'Dounia', '/article[1]'], ['2009011', 'Q0', '10037185', 7, 2.5436, 'Dounia', '/article[1]'], ['2009011', 'Q0', '10045226', 8, 0.6454, 'Dounia', '/article[1]'], ['2009011', 'Q0', '1004679', 9, 3.1454, 'Dounia', '/article[1]'], ['2009011', 'Q0', '1004743', 10, 1.8545, 'Dounia', '/article[1]'], ['2009011', 'Q0', '10048713', 11, 0.6454, 'Dounia', '/article[1]'], ['2009011', 'Q0', '100558', 12, 1.5795, 'Dounia', '/article[1]'], ['2009011', 'Q0', '10056712', 13, 1.0051, 'Dounia', '/article[1]'], ['2009011', 'Q0', '10057294', 14, 2.2352, 'Dounia', '/article[1]'], ['2009011', 'Q0', '1005923', 15, 2.0755, 'Dounia', '/article[1]'],